In [1]:
!pip install chromadb
!pip install pypdf
!pip install pyspellchecker

In [2]:
from pathlib import Path
import os
import shutil

from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings

from langchain.text_splitter import RecursiveCharacterTextSplitter
# from langchain_community.document_loaders import TextLoader

from langchain.document_loaders import PyPDFLoader
from langchain.vectorstores import Chroma

from autocorrect import Speller


In [3]:
SENTENCE_MODEL_NAME = "ai-forever/sbert_large_nlu_ru"
# LLM_MODEL_NAME = "ai-forever/FRED-T5-1.7B"
# LLM_MODEL_NAME = "ai-forever/FRED-T5-large"
# LLM_MODEL_FOLDER = "TheBloke/Llama-2-7B-GGUF"
LLM_MODEL_NAME = "llama-2-7b.Q4_K_M.gguf"

In [4]:
WORKDIR_PATH = Path.home()
DOCS_PATH = WORKDIR_PATH / "data" / "auto_txt_export"
MODEL_PATH = WORKDIR_PATH / "models"
CHROMA_PATH = WORKDIR_PATH / "chromadb"

COLLECTION_NAME = "articles"

In [5]:
# TODO: стоит разобрать как делать кастомные токены и стоит ли
JOIN_TOKEN = "\n\n"
METADATA_TOKEN = "\n"

In [6]:
chunk_size = 1000
chunk_overlap = 300
# set_cache_dir(MODEL_PATH)

# Разворот моделей

## sentencetransformer

In [7]:
embedding_function = SentenceTransformerEmbeddings(model_name=SENTENCE_MODEL_NAME)

/opt/conda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
No sentence-transformers model found with name /home/jovyan/.cache/torch/sentence_transformers/ai-forever_sbert_large_nlu_ru. Creating a new one with MEAN pooling.
/opt/conda/lib/python3.11/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


# Удаление старой vectordb

In [8]:
# shutil.rmtree(CHROMA_PATH)
# os.makedirs(CHROMA_PATH)

# Инициализация векторной db

In [9]:
def load_documents(path = DOCS_PATH):
    # spell = Speller("ru")
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)

    # load the document and split it into chunks
    txt_paths_list = [str(doc) for doc in Path(path).glob("*.txt")]
    # pdf_paths_list = [str(doc) for doc in Path(path).glob("*.pdf")]
    for txt_path in txt_paths_list:
        print(f"current docs: {txt_path}")
        with open(txt_path) as f:
            file = f.read()
            file_name = f.name
        # split it into chunks
        docs = text_splitter.create_documents([file])
        for idx, _ in enumerate(docs):
            docs[idx].metadata = {"file_name": file_name}
        # clean with spellchecker
        # for doc in docs:
        #     doc.page_content = spell(doc.page_content)
        
        # load it into Chroma
        Chroma.from_documents(
            docs,
            embedding_function,
            collection_name=COLLECTION_NAME,
            persist_directory=str(CHROMA_PATH)
        )
    return docs

In [10]:
docs = load_documents()

current docs: /home/jovyan/data/auto_txt_export/7.Нейронные сети и маш.обучение.txt
current docs: /home/jovyan/data/auto_txt_export/6 О подходах к изучению субъективного благополучия личности в зарубежных исследованиях .txt
current docs: /home/jovyan/data/auto_txt_export/5. Мат.модель математ.логики - статья.txt
current docs: /home/jovyan/data/auto_txt_export/6. Диалектика и мат.логика - статья.txt
current docs: /home/jovyan/data/auto_txt_export/3 О роли сознания в субъективном благополучии личности.txt
current docs: /home/jovyan/data/auto_txt_export/11 Субъективное благополучие и эмоциональные переживания в контексте адаптации личности к длительной экстремальной ситуации.txt
current docs: /home/jovyan/data/auto_txt_export/2. Петренко В.Ф. Многомерное сознание.txt
current docs: /home/jovyan/data/auto_txt_export/7 Факторы, определяющие субъективное благополучие личности.txt
current docs: /home/jovyan/data/auto_txt_export/9. ОБЗОР РАЗВИТИЯ ИИ - статья.txt
current docs: /home/jovyan/data/

# Тесты

In [11]:
chroma_db = Chroma(
        persist_directory=str(CHROMA_PATH),
        embedding_function=embedding_function,
        collection_name=COLLECTION_NAME,
    )

similiartiy_responce = chroma_db.similarity_search("благополучие студента", k=5)
beatify2print = JOIN_TOKEN.join(list(map(lambda x: x.page_content + METADATA_TOKEN + str(x.metadata), similiartiy_responce)))
# .join("\n\n")
# similiartiy_pg_cnt
print(beatify2print)

благополучия «Социальное одобрение» (ϕэмп. = 1,6992; р < 0,05), причем более высокие оценки получены у подгруппы с низкими значениями социального капитала, такие студенты особое влияние уделяют обратной связи от тех, с кем они взаимодействуют, ожидают одобрения, поощрения, поддержки для сохранения своей эмоциональной стабильности.
{'file_name': '/home/jovyan/data/auto_txt_export/old.txt'}

благополучия «Социальное одобрение» (ϕэмп. = 1,6992; р < 0,05), причем более высокие оценки получены у подгруппы с низкими значениями социального капитала, такие студенты особое влияние уделяют обратной связи от тех, с кем они взаимодействуют, ожидают одобрения, поощрения, поддержки для сохранения своей эмоциональной стабильности.
{'file_name': '/home/jovyan/data/auto_txt_export/old2.txt'}

благополучия «Социальное одобрение» (ϕэмп. = 1,6992; р < 0,05), причем более высокие оценки получены у подгруппы с низкими значениями социального капитала, такие студенты особое влияние уделяют обратной связи от т